<table>
<tr>
    <td><img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=150 alt="Project Pythia Logo"></img></td>
    <td><img src="images/logos/numpy_logo.svg" width=250 alt="NumPy Logo"></img></td>
    <td><img src="images/logos/xarray_logo.png" width=250 alt="Xarray Logo"></img></td>
</tr>
<tr>
    <td><img src="images/logos/cartopy_logo.png" width=250 alt="Cartopy Logo"></img></td>
    <td><img src="images/logos/geocat_logo.svg" width=300 alt="GeoCAT Logo"></img></td>
    <td><img src="images/logos/matplotlib_logo.svg" width=300 alt="Matplotlib Logo"></img>
</tr></table>

# Matplotlib: Consistant Colorbars for Comparisons

---

## Overview
Analyzing data almost always requires comparing datasets, and when done visually, this is a powerful way to communicate changes in data. However, to do this acurately with color, the colormap used by the plots must be the same. The colors on each plot must have the same meanings or values assigned to them. This cookbook shows how to:

1. create multiple plots but display only one colorbar
1. use the same colormap for multiple plots
1. choose accessible colormaps and explains best practices when using color

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Helpful | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Opening data sets |
| [Matplotlib Basics](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html#overview) | Necessary | [Contour/Filled Contour Plots](https://foundations.projectpythia.org/core/matplotlib/matplotlib-basics.html#contour-and-filled-contour-plots) |
| [Basic Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html#opening-netcdf-data) | Necessary | [Opening netCDF data](https://foundations.projectpythia.org/core/xarray/xarray-intro.html#opening-netcdf-data) |
| [Basic Numpy](https://foundations.projectpythia.org/core/numpy/numpy-basics.html) | Necessary | |
| [GeoCAT-viz](https://geocat-viz.readthedocs.io/en/latest/) | Helpful | For formatting plots |
| [GeoCAT-datafiles](https://github.com/ncar/geocat-datafiles) | Helpful | |

- **Time to learn**: 15 minutes

---

## Getting set up
### Imports

In [9]:
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import geocat.datafiles as gdf
import geocat.viz as gv
%matplotlib inline

### Loading in and preprocessing data

The data we are using for this example is from the [GeoCAT-datafiles](https://github.com/NCAR/geocat-datafiles) repository on GitHub. The specific file is `TS.cam3.toga_ENS.1950-2000.nc`, and it contains surafce temeprature data from the first run of the CAM3 T85 TOGA model run. We load the data file in using the xarray default engine.

In [ ]:
ds = xr.open_dataset(gdf.get("netcdf_files/TS.cam3.toga_ENS.1950-2000.nc"),)
ds

The file has data from different timesteps of the model. Say we want to plot temperature data from two different timesteps and compare them visually. To see the biggest change, let's choose a timestep from February and one from August in the same year. We also need to select the variable of interest which is `TS` for surface temperature.

To create the colorbar, we'll need to know what the minimum and maximum temperatures are. We find those here as well.

In [11]:
# Select the timesteps
first_step = ds.isel(time=0).TS
last_step = ds.isel(time=6).TS

# Find temperature min/max
min_val = np.floor(min([first_step.min(), last_step.min()]).values)
max_val = np.ceil(max([first_step.max(), last_step.max()]).values)

## Making the Plot
Due to Jupyter Notebook limitations, the entire plot must be made in the same block. These are the general steps:

### Create the figure

We are using `matplotlib` and `cartopy` to make two plots on the same figure. We need to create the figure and specify our map projection before we can create the two subplots.

### Format and populate subplots

Once that's done, we need to format and populate the subplots. The function `create_subplot` below ensures that both sets of axes are created with the same specifications. *Most importantly, it makes sure that both plots are drawn with the same `levels` keyword for `contourf`.* This keyword is what specifies how the colors are assigned to values. As long as `levels` is the same between contour plots, the images will be useful for making accurate comparisions.

### Making the colorbar

`contourf` returns a `QuadContourSet` object. This holds the information about how colors are assigned to values and where the contours are drawn. Since both plots were made with the same parameters, the returned objects are identical in regards to color assignments. Either can be passed into `plt.colorbar`.

To make the colorbar appear below both plots, the `ax` parameter must be set to the lower axes `ax2`. Then `orientation` is set to `'horizontal'`. Additional [keyword arguments](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html#matplotlib.pyplot.colorbar) can be passed in to `plt.colorbar` to further customize its appearance.

In [ ]:
# Helper function to format and populate subplots
def create_plot(ax, data, cmap):
    # Use geocat.viz.util convenience function to set axes tick values
    gv.set_axes_limits_and_ticks(ax=ax,
                                 xlim=(-180, 180),
                                 ylim=(-90, 90),
                                 xticks=np.linspace(-180, 180, 13),
                                 yticks=np.linspace(-90, 90, 7))

    # Use geocat.viz.util convenience function to make plots look like NCL
    # plots by using latitude, longitude tick labels
    gv.add_lat_lon_ticklabels(ax)

    # Remove the degree symbol from tick labels
    ax.yaxis.set_major_formatter(LatitudeFormatter(degree_symbol=''))
    ax.xaxis.set_major_formatter(LongitudeFormatter(degree_symbol=''))

    # Use geocat.viz.util convenience function to add minor and major ticks
    gv.add_major_minor_ticks(ax)

    # Draw coastlines
    ax.coastlines(linewidth=0.5)

    # Use geocat.viz.util convenience function to set titles
    gv.set_titles_and_labels(ax,
                             maintitle=data.time.values,
                             lefttitle='TS',
                             righttitle='K',
                             maintitlefontsize=12,
                             lefttitlefontsize=10,
                             righttitlefontsize=10)
    
    return ax.contourf(data['lon'], data['lat'], data.data, cmap=cmap, levels=np.linspace(min_val, max_val, 10))

# Creating the figure and subplots
fig = plt.figure(figsize=(24, 8), constrained_layout=True)
proj = ccrs.PlateCarree()
ax1 = fig.add_subplot(2,1,1, projection=proj)  # Top axes
ax2 = fig.add_subplot(2,1,2, projection=proj)  # Bottom axes

# Plotting data
mappable = create_plot(ax1, first_step, 'jet')
mappable = create_plot(ax2, last_step, 'jet')

# Making colorbar
cbar = fig.colorbar(mappable, ax=ax2, orientation='horizontal', aspect=13)

## Colormap Best Practices
Color is a useful tool for communicating information. However, there are important factors to consider when choosing color palettes that may not be obvious at first.

### Printing in Black and White
If your plot is printed in black and white, there can be issues telling the colors apart and interpreting the plot. This is because the colors are reduced to their brightness. Colormaps have a variety of hues and levels of saturation to communicate information, but not all of them are designed to have linear changes in brightness for black and white printing. Below is an example of how the rainbow colormap we used before is hard to interpret in black and white.

<table><tr><td>
<img src="images/jet.png" width=100% alt="Two world maps with temperature contours. The contours are drawn with a rainbow color palette."></img></td>
<td>
    
<img src="images/jet_bw.png" width=100% alt="Two world maps with temperature contours. The image has been converted to grayscale."></img></td></tr></table>

### Choosing Accessible Color Palettes
Colorblindness affects approximately 1 in 12 men and 1 in 200 women [[1]](https://www.colourblindawareness.org/colour-blindness/). To make scientific information accessible to everyone, how different kinds of colorblindness affect the interpretation of color schemes must be taken into account.

Below is an approximation of how someone with deuteranomaly colorblindness would see the plot we created. This kind of color deficiency can be described as "green weakness" since the green receptors in the eyes exist but have differences in how sensitive they are to green light. This is the most common kind of colorblindness, but there are many other types [[2]](https://www.color-blindness.com/types-of-color-blindness/). 

Note that some of the colors are now hard to distinguish and the plot is harder to interpret [[3]](https://www.color-blindness.com/coblis-color-blindness-simulator/).
<table><tr><td>
<img src="images/jet.png" width=100% alt="Two world maps with temperature contours. The contours are drawn with a rainbow color palette."></img></td>
<td>
    
<img src="images/jet_deuteranomaly.png" width=100% alt="Two world maps with temperature contours. The image has been modified to simulate deuteranomaly color deficiency."></img></td></tr></table>

Recommended colormaps for sequential data are `viridis`, `plasma`, `inferno`, `magma`, and `cividis`. These colormaps increase linearly in brightness, thus the actual hue of the colors is not the only way to distinguish them. Below is an example of the `plasma` colormap and how it looks in simulated deuteranomaly and in grayscale. This colormap is still easily interpretable by someone with deuteranomaly and in grayscale printing.

<table><tr><td>
<img src="images/plasma.png" width=100% alt="Two world maps with temperature contours. The contours are drawn with a rainbow color palette."></img></td></tr>
<tr><td><img src="images/plasma_deuteranomaly.png" width=150% alt="Two world maps with temperature contours. The image has been modified to simulate deuteranomaly color deficiency."></img></td><td><img src="images/plasma_bw.png" width=150% alt="Two world maps with temperature contours. The image has been converted to grayscale."></img></td>
</tr></table>

### Alternative Color Palettes
Matplotlib has an [entire lesson](https://matplotlib.org/stable/tutorials/colors/colormaps.html) explaining the different classes of colormaps and the appropriate times to use them. It is a more in depth resource for using color than this cookbook and is highly recommended if you want more information on this topic.

---

## Summary
Now that you have finished going through this cookbook, you should be able to create multiple plots that use the same colormap and colorbar. Additionally, you should know how colorblindness can affect the interpretation of colormaps and which colormaps are more accessible.

## Resources and references
For more information on choosing colormaps in Matplotlib, see their [lesson on colormaps.](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

For more examples of accessible colormaps in scientific visualizations, check out [the color example scripts](https://geocat-examples.readthedocs.io/en/latest/gallery/index.html#colors) on the GeoCAT-examples gallery.

###
Cartopy. v0.20.3. 29-Jun-2022. Met Office. UK. [doi:10.5281/zenodo.6775197](https://doi.org/10.5281/zenodo.6775197).

Harris, C.R., Millman, K.J., van der Walt, S.J. et al. Array programming with NumPy. Nature 585, 357–362 (2020). [doi: 10.1038/s41586-020-2649-2](https://doi.org/10.1038/s41586-020-2649-2).

J. D. Hunter, "Matplotlib: A 2D Graphics Environment", Computing in Science & Engineering, vol. 9, no. 3, pp. 90-95, 2007. v3.5.2. [doi:10.5281/zenodo.6513224](https://doi.org/10.5281/zenodo.6513224)

Hoyer, S., Fitzgerald, S., Hamman, J. et al. (2022). xarray (v2022.03.0) [Software]. [doi:10.5281/zenodo.6323468](https://doi.org/10.5281/zenodo.6323468)

Visualization & Analysis Systems Technologies. (2022). Geoscience Community Analysis Toolkit: GeoCAT-datafiles (v2022.03.0) [Software]. Boulder, CO, USA: UCAR/NCAR - Computational and Informational System Lab. [doi:10.5281/zenodo.6684830](https://doi.org/10.5281/zenodo.6684830).

Visualization & Analysis Systems Technologies. (2022). Geoscience Community Analysis Toolkit: GeoCAT-viz (v2022.05.0) [Software]. Boulder, CO, USA: UCAR/NCAR - Computational and Informational System Lab. [doi:10.5281/zenodo.6678351](https://doi.org/10.5281/zenodo.6678351).